<a href="https://colab.research.google.com/github/rudraser/Fluter-BasicApps/blob/main/Custom_ResNet_attempt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

In [ ]:

from matplotlib import pyplot as plt
import numpy as np
import collections


import torch.nn as nn

import torch.optim as optim

from torchvision import datasets, transforms
data_path = '/data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(size=[32,32], padding=4),                        
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [ ]:
from torchvision import datasets, transforms
data_path = '/data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(size=[32,32], padding=4),                        
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [ ]:
class block(nn.Module):
    def __init__(self, in_channels)
        super(block, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU()
        
    def forward(self,x)
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        return x + out


In [ ]:
class ResNet(nn.Module)
    def __init__(self, in=48, n_blocks=8):
        super().__init__()
        self.in = in
        self.conv1 = nn.Conv2d(3, in, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in, 2*in, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(2*in, 4*in, kernel_size=3, padding=1)       # defing layers for forward
        self.resblocks = nn.Sequential(
            *(n_blocks * [resblock(n=in)]))
        self.resblocks2 = nn.Sequential(
            *((n_blocks*3) * [resblock(n=in*2)]))
        self.resblocks3 = nn.Sequential(
            *(n_blocks * [resblock(n=in*4)]))
        self.fc1 = nn.Linear(8 * 8 * in*4, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 10)

    def forward(self, x):                    # using the resblocks 3 times, each with twice the number of channels as before
        x = torch.relu(self.conv1(x))
        x = self.resblocks(x)                          
        x = F.max_pool2d(x, 2) 
        x = self.conv2(x)
        x = self.resblocks2(x)
        x = F.max_pool2d(x, 2) 
        x = self.conv3(x)
        x = self.resblocks3(x)
        x = x.view(-1, 8 * 8 * self.in*4)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):           # defining the training loop
    for epoch in range(1, n_epochs + 1):                                        # these inputs would be defined in next cell
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)  # <1>
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        
        print('{} Epoch {}, Training loss {}'.format(
            datetime.datetime.now(), epoch,
            loss_train / len(train_loader)))

In [ ]:

train_loader = torch.utils.data.DataLoader(cifar10, batch_size=32,
                                           shuffle=True)                   

model =  deep().to(device=device)# <1>
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=0.01)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
training_loop(                   #calling the training loop finally
    n_epochs = 60,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():   # we do no need backpropagation for this hence the nograd
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # taking the class at maximum probability as prediction
                total += labels.shape[0]  
                correct += int((predicted == labels).sum())   # calculating the no. of correct predictions.

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)